In [54]:
import pandas as pd
import polars as pl
import os
from functions.fix_parquet_with_header_data import fix_parquet_with_header_data
from functions.prepare_electric_filenames_from_household_sensors import prepare_electric_filenames_from_household_sensors
from functions.aggregate_data_daily import aggregate_data_daily
from functions.prepare_electric_filenames_from_appliance import prepare_electric_filenames_from_appliance
import matplotlib.pyplot as plt
from pathlib import Path

### Przygotowanie df-a z lokalizacjami plików do agregowania

In [2]:
folder_path = "../../Data/Parquet/household_sensors/sensordata"
household_sensors_data_to_process = prepare_electric_filenames_from_household_sensors(folder_path)

In [3]:
folder_path = "../../Data/Parquet/room_and_appliance_sensors/sensordata"
appliance_data_to_process = prepare_electric_filenames_from_appliance(folder_path)

In [4]:
len(household_sensors_data_to_process["home_id"].unique())

254

In [5]:
appliance_data_to_process

,home_id,electric,appliance_name,filename
0,61,electric-appliance,kettle,home61_kitchen695_sensor1967_electric-applianc...
1,61,electric-appliance,dishwasher,home61_kitchen695_sensor1969_electric-applianc...
2,61,electric-appliance,microwave,home61_kitchen695_sensor1971_electric-applianc...
3,62,electric-appliance,fridgefreezer,home62_kitchen710_sensor1779_electric-applianc...
4,62,electric-appliance,kettle,home62_kitchen710_sensor1780_electric-applianc...
...,...,...,...,...
212,311,electric-appliance,vacuumcleaner,home311_kitchen2857_sensor31438_electric-appli...
213,328,electric-appliance,kettle,home328_kitchen2980_sensor31003_electric-appli...
214,328,electric-appliance,toaster,home328_kitchen2980_sensor31004_electric-appli...
215,328,electric-appliance,fridgefreezer,home328_kitchen2980_sensor31005_electric-appli...


In [6]:
household_sensors_data_to_process

,home_id,electric,subcircuit,filename
0,47,electric-mains_electric-combined,None,home47_hall654_sensor1216c1220_electric-mains_...
1,59,electric-mains_electric-combined,None,home59_hall687_sensor1506c1508_electric-mains_...
2,61,electric-mains_electric-combined,None,home61_hall693_sensor1563c1569_electric-mains_...
3,61,electric-subcircuit,mains,home61_utility1109_sensor5071_electric-subcirc...
4,62,electric-mains_electric-combined,None,home62_hall705_sensor1662c1666_electric-mains_...
...,...,...,...,...
329,331,electric-mains_electric-combined,None,home331_hall2999_sensor21563c21567_electric-ma...
330,332,electric-mains_electric-combined,None,home332_hall3027_sensor21901c21903_electric-ma...
331,333,electric-mains_electric-combined,None,home333_hall3015_sensor21748c21765_electric-ma...
332,334,electric-mains_electric-combined,None,home334_hall3013_sensor21787c21791_electric-ma...


### Poprawienie pierwszej kolumny i agregacja dancyh do dziennych - przykład

In [7]:
df = pl.read_parquet("../../Data/Parquet/room_and_appliance_sensors/sensordata/home61_kitchen695_sensor1967_electric-appliance_kettle.parquet")
df_clean = fix_parquet_with_header_data(df)
daily_kettle = aggregate_data_daily(df_clean)

In [8]:
# zaczytanie danych o domu
df_home_id=pl.read_csv('../../Data/NAV/Features_HomeID.csv')
df_home_id

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


### appliance_data_to_process

In [9]:
# Zmiana appliance_data_to_process na DataFrame Polars
appliance_data_to_process_pl = pl.DataFrame(appliance_data_to_process)
appliance_data_to_process_pl

home_id,electric,appliance_name,filename
i64,str,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…"
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…"
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…"
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…"
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…"
…,…,…,…
311,"""electric-appliance""","""vacuumcleaner""","""home311_kitchen2857_sensor3143…"
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…"
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…"


In [10]:
df_joined = appliance_data_to_process_pl.join(
    df_home_id.unique(subset=["homeid_1"]),
    left_on='home_id',
    right_on='homeid_1',
    how='left'
)
df_joined

home_id,electric,appliance_name,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
311,"""electric-appliance""","""vacuumcleaner""","""home311_kitchen2857_sensor3143…",1,1,2,0,2,0,null,null,8,"""Don't own""","""flat"""
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [11]:
df_all = []

# Iterujemy po unikalnych wierszach z potrzebnymi informacjami
for row in df_joined.iter_rows(named=True):
    home_id = row["home_id"]
    filename = row["filename"]
    device = row["appliance_name"]

    path_file = Path(f"../../Data/Parquet/room_and_appliance_sensors/sensordata/{filename}")
    # print(path_file)

    try:
        df = pl.read_parquet(path_file)
    
        if df.is_empty():
            print(f"POMINIĘTO pusty plik: {path_file}")
            continue

        df_clean = fix_parquet_with_header_data(df)
        daily_data = aggregate_data_daily(df_clean)

        df_tmp = daily_data.with_columns([
            pl.lit(home_id).alias("home_id"),
            pl.lit(device).alias("device")
        ])

        df_all.append(df_tmp)

    except Exception as e:        
        print(f"Błąd przy pliku {path_file}: {e}")
        continue

# Łączenie wszystkich przetworzonych ramek
df_all = pl.concat(df_all)

In [12]:
df_all

date,daily_sum,home_id,device
date,i64,i32,str
2016-11-17,11763,61,"""kettle"""
2016-11-18,14256,61,"""kettle"""
2016-11-19,22827,61,"""kettle"""
2016-11-20,25511,61,"""kettle"""
2016-11-21,19826,61,"""kettle"""
…,…,…,…
2018-05-26,3776,328,"""microwave"""
2018-05-27,0,328,"""microwave"""
2018-05-28,2662,328,"""microwave"""


In [13]:
# df_joined.filter(pl.col('home_id') == 328)['filename'][-1]
# path_file = Path(f"../../Data/Parquet/room_and_appliance_sensors/sensordata/{df_joined.filter(pl.col('home_id') == 328)['filename'][-1]}")
# path_file
# clean_microwave = fix_parquet_with_header_data(pl.read_parquet(path_file))
# daily_microwave = aggregate_data_daily(clean_microwave)
# daily_microwave

### Dojoinowanie danych o elekryczności z appliance do danych o domu

In [14]:
df_all

date,daily_sum,home_id,device
date,i64,i32,str
2016-11-17,11763,61,"""kettle"""
2016-11-18,14256,61,"""kettle"""
2016-11-19,22827,61,"""kettle"""
2016-11-20,25511,61,"""kettle"""
2016-11-21,19826,61,"""kettle"""
…,…,…,…
2018-05-26,3776,328,"""microwave"""
2018-05-27,0,328,"""microwave"""
2018-05-28,2662,328,"""microwave"""


In [15]:
df_all = df_all.with_columns([
    pl.col("device").str.strip_chars().str.to_lowercase()
])

df_joined = df_joined.with_columns([
    pl.col("appliance_name").str.strip_chars().str.to_lowercase()
])

df_panel_long = df_all.join(
    df_joined,
    left_on=["home_id", "device"],
    right_on=["home_id", "appliance_name"],
    how="left"
)

In [16]:
df_panel_long

date,daily_sum,home_id,device,electric,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
date,i64,i32,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
2016-11-17,11763,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-18,14256,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-19,22827,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-20,25511,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-21,19826,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-05-26,3776,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-27,0,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-28,2662,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [92]:
df_panel_long.write_parquet("../../Data/ML_DF/data_frame_panel_long.parquet")

In [93]:
pivot_df = df_panel_long.pivot(
    values="daily_sum",
    index=["date",'home_id'],
    columns="device",
    aggregate_function='first'  # or "sum" if you expect multiple entries per date/device
)

C:\Users\jansl\AppData\Local\Temp\ipykernel_19544\2203284142.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivot_df = df_panel_long.pivot(


In [94]:
df_X=df_home_id.join(
    pivot_df,
    left_on='homeid_1',
    right_on='home_id',
    how='left')
df_X.write_parquet("../../Data/ML_DF/data_frame_X.parquet")

In [95]:

df_X = df_X.with_columns([
    (pl.col("smart_automation") != "Don't own").cast(pl.Int8).alias("smart_automation_binary")
])
df_X=df_X.drop("smart_automation")
df_X = df_X.to_dummies(columns=["hometype"])
df_X=df_X.drop('hometype_house_or_bungalow')

In [96]:
df_X.head()

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,hometype_flat,date,kettle,dishwasher,microwave,fridgefreezer,washingmachine,vacuumcleaner,washingmachinetumbledrier,other,toaster,freezer,fridge,dehumidifier,electricheater,tumbledrier,grill,appliance,electricoven,smart_automation_binary
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,u8,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i8
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-17,11763,2969,342,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-18,14256,14482,9147,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-19,22827,7705,4858,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-20,25511,20033,3333,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-21,19826,133,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0


In [97]:
df_X.select([
    pl.col(col).is_null().sum().alias(col)
    for col in df_X.columns
])


homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,hometype_flat,date,kettle,dishwasher,microwave,fridgefreezer,washingmachine,vacuumcleaner,washingmachinetumbledrier,other,toaster,freezer,fridge,dehumidifier,electricheater,tumbledrier,grill,appliance,electricoven,smart_automation_binary
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,5226,5226,0,0,0,18091,19137,19765,17593,18356,35016,35460,39299,31453,37647,37351,37972,39913,40339,42154,42054,42295,164


In [107]:
df_without_clmns=df_X.drop(['grill','appliance','tumbledrier','electricheater','dehumidifier','other','fridge','freezer','washingmachinetumbledrier','electricoven','toaster','vacuumcleaner','fridgefreezer']) # electricoven	 bylo usuniete

In [104]:
df_without_clmns.select([
    pl.col(col).is_null().sum().alias(col)
    for col in df_without_clmns.columns
])

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,hometype_flat,date,kettle,dishwasher,microwave,fridgefreezer,washingmachine,smart_automation_binary
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,5226,5226,0,0,0,18091,19137,19765,17593,18356,164


In [106]:
df_without_clmns

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,hometype_flat,date,kettle,dishwasher,microwave,fridgefreezer,washingmachine,smart_automation_binary
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,u8,date,i64,i64,i64,i64,i64,i8
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-17,11763,2969,342,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-18,14256,14482,9147,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-19,22827,7705,4858,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-20,25511,20033,3333,null,null,0
61,1,1,2,0,2,0,0.33,0.12,9,0,2016-11-21,19826,133,4,null,null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,1,2018-05-26,8791,null,3776,20097,null,0
328,0,1,1,0,0,1,null,null,6,1,2018-05-27,12940,null,0,16200,null,0
328,0,1,1,0,0,1,null,null,6,1,2018-05-28,2934,null,2662,18679,null,0


In [110]:
df_appl_without_null=df_without_clmns.drop_nulls()

In [112]:
df_appl_without_null.write_parquet('../../Data/ML_DF/df_appl_without_null.parquet')

### household_sensors_data_to_process

In [20]:
# Zmiana appliance_data_to_process na DataFrame Polars
household_sensors_data_to_process = pl.DataFrame(household_sensors_data_to_process)
household_sensors_data_to_process

home_id,electric,subcircuit,filename
i64,str,str,str
47,"""electric-mains_electric-combin…",null,"""home47_hall654_sensor1216c1220…"
59,"""electric-mains_electric-combin…",null,"""home59_hall687_sensor1506c1508…"
61,"""electric-mains_electric-combin…",null,"""home61_hall693_sensor1563c1569…"
61,"""electric-subcircuit""","""mains""","""home61_utility1109_sensor5071_…"
62,"""electric-mains_electric-combin…",null,"""home62_hall705_sensor1662c1666…"
…,…,…,…
331,"""electric-mains_electric-combin…",null,"""home331_hall2999_sensor21563c2…"
332,"""electric-mains_electric-combin…",null,"""home332_hall3027_sensor21901c2…"
333,"""electric-mains_electric-combin…",null,"""home333_hall3015_sensor21748c2…"


In [21]:
df_all_sensors = []

# Iterujemy po unikalnych wierszach z potrzebnymi informacjami
for row in household_sensors_data_to_process.iter_rows(named=True):
    home_id = row["home_id"]
    filename = row["filename"]
    subcircuit = row["subcircuit"]

    path_file = Path(f"../../Data/Parquet/household_sensors/sensordata/{filename}")
    # print(path_file)

    try:
        df = pl.read_parquet(path_file)
    
        if df.is_empty():
            print(f"POMINIĘTO pusty plik: {path_file}")
            continue

        df_clean = fix_parquet_with_header_data(df)
        daily_data = aggregate_data_daily(df_clean)

        df_tmp = daily_data.with_columns([
            pl.lit(home_id).alias("home_id"),
            pl.lit(subcircuit).alias("subcircuit")
        ])

        if not df_tmp.is_empty():
            df_all_sensors.append(df_tmp)
        else:
            print(f"⚠️ Pusty wynik po agregacji dla: {path_file}")
            
        df_all_sensors.append(df_tmp)

    except Exception as e:      
        print(f"Błąd przy pliku {path_file}: {e}")
        continue

KeyboardInterrupt: 

In [59]:
for i in range(len(df_all_sensors)):
    if "subcircuit" not in df_all_sensors[i].columns:
        df_all_sensors[i] = df_all_sensors[i].with_columns(pl.lit("").alias("subcircuit"))
    else:
        df_all_sensors[i] = df_all_sensors[i].with_columns(df_all_sensors[i]["subcircuit"].cast(pl.Utf8))

df_all_sensors = pl.concat(df_all_sensors)

In [ ]:
#df_all_sensors.write_parquet("../../Data/ML_DF/df_all_sensors.parquet")

In [63]:
household_sensors_data_to_process

home_id,electric,subcircuit,filename
i64,str,str,str
47,"""electric-mains_electric-combin…",null,"""home47_hall654_sensor1216c1220…"
59,"""electric-mains_electric-combin…",null,"""home59_hall687_sensor1506c1508…"
61,"""electric-mains_electric-combin…",null,"""home61_hall693_sensor1563c1569…"
61,"""electric-subcircuit""","""mains""","""home61_utility1109_sensor5071_…"
62,"""electric-mains_electric-combin…",null,"""home62_hall705_sensor1662c1666…"
…,…,…,…
331,"""electric-mains_electric-combin…",null,"""home331_hall2999_sensor21563c2…"
332,"""electric-mains_electric-combin…",null,"""home332_hall3027_sensor21901c2…"
333,"""electric-mains_electric-combin…",null,"""home333_hall3015_sensor21748c2…"


In [76]:
# # Unikalne wartości dla electric subcircuitów

# unique_subcircuits = (
#     household_sensors_data_to_process
#     .filter(pl.col("electric") == "electric-subcircuit")
#     .select("subcircuit")
#     .unique()
# )

# print(unique_subcircuits)

In [ ]:
# SPrawdzenie czy dla jednego home_id jest więcej niż jeden subcircuit dla electric-subcircuit - nie ma
(
    household_sensors_data_to_process
    .filter(pl.col("electric") == "electric-mains_electric-combined")
    .group_by("home_id")
    .agg(pl.count().alias("count_combined"))
    .filter(pl.col("count_combined") > 1)
)

/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_68391/2697917958.py:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("count_combined"))


home_id,count_combined
i64,u32


In [74]:
# zamiana nulli w kolumnie subcircuit na "total_power_consumption"
df_all_sensors = df_all_sensors.with_columns(
    pl.when(pl.col("subcircuit").is_null())
    .then(pl.lit("total_power_consumption"))
    .otherwise(pl.col("subcircuit"))
    .alias("subcircuit")
)

In [78]:
pivot_sensors = df_all_sensors.pivot(
    values="daily_sum",
    index=["date",'home_id'],
    columns="subcircuit",
    aggregate_function='first'  # or "sum" if you expect multiple entries per date/device
)

/var/folders/rh/w6cl6rd95vj2g4s03scz_4fc0000gn/T/ipykernel_68391/200684759.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivot_sensors = df_all_sensors.pivot(


In [82]:
df_energy_home = df_home_appliances.join(
    pivot_sensors,
    left_on=['homeid_1', 'date'],
    right_on=['home_id', 'date'],
    how='left'
)

In [84]:
df_energy_home.write_parquet("../../Data/ML_DF/df_energy_home.parquet")